<a href="https://colab.research.google.com/github/eliransr/deepLearning_models/blob/master/Bears%20classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

###Creating your own dataset from Google Images
###Create directory and upload urls files into my server
from fastai import *
from fastai.vision import *

In [0]:
###Get a list of URLs
##Go to Google Images and search for the images you are interested in.
##Download into file
# urls=Array.from(document.querySelectorAll('.rg_i')).map(el=> el.hasAttribute('data-src')?el.getAttribute('data-src'):el.getAttribute('data-iurl'));
# window.open('data:text/csv;charset=utf-8,' + escape(urls.join('\n')));

In [0]:
##Create directory and upload urls file into your server
folder = 'black'
file = 'black_bears.txt'

In [0]:
folder = 'grizzly'
file = 'brown_bears.txt'

In [0]:
folder = 'teddys'
file = 'teddys.txt'

In [0]:
path = Path('data/bears')
dest = path/folder
dest.mkdir(parents=True , exist_ok=True)

In [0]:
##Download images
classes = ['teddys' , 'grizzly' , 'black']

In [20]:
download_images(path/file,dest, max_pics=200)

FileNotFoundError: ignored

In [10]:
for c in classes:
  print(c)
  verify_images(path/c, delete=True, max_workers=8)

teddys
grizzly
black


In [11]:
##View data
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
       ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:458: UserWarning: Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.
  warn("Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.")
/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:461: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


Exception: ignored

In [12]:
data.classes

NameError: ignored

In [0]:
data.show_batch(rows=3, figsize=(7,8))

SyntaxError: ignored